In [5]:
def is_colab():
    """Check if the code is running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Use the function to conditionally run magic commands
if is_colab():
    # Run Colab-specific magic commands
    print("Running in Colab, executing magic commands.")
    !rm -rf microproyecto3NLP/
    !git clone https://github.com/cjohana031/microproyecto3NLP
    !cp -R microproyecto3NLP/* .
    # Add any other Colab-specific setup
else:
    # Alternative setup for non-Colab environments
    print("Not running in Colab, nothing else is needed.")

Running in Colab, executing magic commands.
Cloning into 'microproyecto3NLP'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 34.19 MiB | 33.03 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [6]:
with open('data/test.txt', 'r') as f:
  print(f.read())

holii


In [11]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [14]:
# Importación de librerías
import sentencepiece as spm
import io
import time
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import BertForQuestionAnswering

# Configurar semillas para facilitar la reproducibilidad de los resultados
seed = 99
torch.manual_seed(seed)
np.random.seed(seed)

In [24]:
#read dataset

dataset_file_path = 'data/test-00000-of-00001.parquet'
test = pd.read_parquet(dataset_file_path)
test = test[test['article'].str.len() < 800]
test



,example_id,article,answer,question,options
45,high18377.txt,A new law helps people with disabilities. The ...,D,According to the passage we can see that_.,[it will be difficult for the normal persons t...
46,high18377.txt,A new law helps people with disabilities. The ...,C,Ramps can help people_.,"[with hearing problems, who have difficulty in..."
47,high18377.txt,A new law helps people with disabilities. The ...,D,"The word ""Braille"" means_.","[words known by foreigners, words known by com..."
48,high18377.txt,A new law helps people with disabilities. The ...,C,What do you think the disabled would feel abou...,"[They may think the new law reasonable., Some ..."
49,high18377.txt,A new law helps people with disabilities. The ...,B,The fact that the new law has been passed show...,[the government has paid no attention to the d...
...,...,...,...,...,...
4915,middle926.txt,"Most People don't like mice, but they love one...",C,"In the cartoon, Pluto is a _ dog.","[clever, smart, foolish, bad]"
4916,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,A,Who is the oldest of the four boys?,"[Dick., David., Dean., Denis.]"
4917,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,D,What do the four boys like doing every day?,"[Reading stories., Playing computer games., Wa..."
4918,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,C,What will they do on Mrs. Black's birthday?,"[Eat out., Play soccer., Have a party., Play c..."
